## Desafío Mercado Libre

### 2. ¿Cuán rápido puedes ordenar estos productos?

Elaborado por : Laura Nogues

#### Descripción

La seccion de ofertas https://www.mercadolibre.com.ar/ofertas#nav-header es una
sección que agrupa las mejores ofertas de MELI, y a grandes rasgos es un listado de
productos en oferta ordenados por un score de ML y distintas reglas de negocio.
Anexado al desafío, se encuentra un archivo “ordenamiento.csv” el cual tiene un
listado de productos, con su score y categorías para este desafío.

El score determina cuán bueno es un item_id, siendo 1 el mejor valor posible.
De no existir reglas de negocio que ponen restricciones sobre el ordenamiento, la
solución ideal sería ordenar nuestro dataset de mayor a menor por score.

El problema es que este ordenamiento debe reflejar variedad de productos y
categorías permitiendo el discovery de distintos tipos de productos por lo cual
tenemos en producción reglas como las siguientes:

1. El “domain_id” no se puede repetir en 4 posiciones consecutivas.
2. El “vertical” no se puede repetir en 1 posición consecutiva.
3. De existir el id 641416750 en el listado debe estar en la posición 3 siendo esta regla
más fuerte que las demás.
4. De existir el id 22351223 en el listado debe estar en la posición 6 siendo esta regla
más fuerte que las demás.
5. Las posiciones 9,10,11 deben tener sí o sí items de la categoría “HOME&DECO”
siendo esta regla más fuerte que la 1 y 2.
6. Cumpliendo estas condiciones, el ordenamiento debe respetar un ordenamiento de
mayor score a menor.
El desafío es diseñar un algoritmo que, dado el dataset y estas restricciones, devuelva
el listado final ordenado de ítems. El algoritmo debe estar diseñado para escalar
eficientemente con el número de ítems, y contemplar los casos en que no se pueden
cumplir las restricciones. ¡El tiempo de ejecución es el factor clave!


**Disclaimer**: Por cuestiones de tiempo el algoritmo no fue finalizado de forma correcta ya que quedo reducido en dos funciones separadas, cuando deberian estar unidas. 


In [2]:
import pandas as pd
import timeit
import time
import numpy as np
from itertools import groupby
import re

In [3]:
def header(df): 
    start = time.time()
    
    df.sort_values(by='score', ascending= False, inplace=True)
    df.reset_index(inplace=True)

    #usamos un dict para mapear los nombres de los productos a numeros creando la columna domain_num
    keys = list(df.domain.unique())
    values = list(range(len(keys)))
    dic = dict(zip(keys, values))
    df['domain_num']= df['domain'].map(dic)
    
   
    def cuatro_consec(df):
        count= 1
        for i in range(len(df)-1): 
            if df.loc[i].domain_num== df.loc[i+1].domain_num: 
                count +=1
                if count >=3:
                    if df.loc[i+2].domain_num== df.loc[i+3].domain_num:#si hay 5 valores iguales consecutivos
                        if df.loc[i+3].domain_num== df.loc[i+4].domain_num:# si hay 6 valores iguales consecutivos
                            df.iloc[i+2], df.iloc[i+3], df.iloc[i+4], df.iloc[i+5]=  df.iloc[i+3],df.iloc[i+4],df.iloc[i+5], df.iloc[i+2]
                        else: 
                            df.iloc[i+2], df.iloc[i+3], df.iloc[i+4]=  df.iloc[i+3],df.iloc[i+4], df.iloc[i+2] #si son 5 muevo los siguientes 2 una posicion
                    else: 
                        df.iloc[i+2], df.iloc[i+3]= df.iloc[i+3], df.iloc[i+2] #si no hay mas de 4 valores consecutivos iguales  
        return df  

    df= cuatro_consec(df)
        

    # De existir el id 641416750 en el listado debe estar en la posición 3 
    # De existir el id 22351223 en el listado debe estar en la posición 6
    for i in range(len(df)-1): 
        if df.loc[i].item_id == 641416750: 
            df.loc[3], df.loc[i]= df.loc[i], df.loc[3]
        if df.loc[i].item_id == 22351223:
            df.loc[6], df.loc[i]= df.loc[i], df.loc[6]
            
      
    df.reset_index(inplace=True)
    
    stop = timeit.default_timer()
    print(f'Time: {time.time() - start} seconds')
    return df

In [4]:
df= pd.read_csv('data/ordenamiento.csv')

In [5]:
df= header(df)

Time: 2.160386562347412 seconds


In [6]:
def dos_consec_vertical(df): 
    def move_row (df, target_row, posicion): 
        df=df.copy()
        resta= posicion - 1
        idx =[i for i in range(posicion-resta)] + [target_row] + [i for i in range(len(df)-posicion) if i != target_row]
        df= df.reindex(idx)
        df.drop(columns='level_0', inplace=True)
        df.reset_index(inplace= True)
        return df

    for i in range(len(df)-1): 
        j = [sum(1 for i in g) for k,g in groupby(df[i:].vertical) if k==df.loc[i].vertical]
        k = j[0]
        if k > 1: 
            target_row= i + k 
            posicion = i + 1 
            try: 
                if not df.iloc[target_row].vertical == df.iloc[i].vertical: 
                    df= move_row(df, target_row, posicion)
            except: 
                break 

    return list(df.domain)
  

In [7]:
lista_domain= dos_consec_vertical(df)

In [8]:
lista_domain

['MLC-GAME_CONSOLES',
 'MLC-OFFICE_CHAIRS',
 'MLC-GAME_CONSOLES',
 'MLC-SURGICAL_AND_INDUSTRIAL_MASKS',
 'MLC-GAME_CONSOLES',
 'MLC-HEALTH_CARE_SUPPLIES',
 'MLC-GAME_CONSOLES',
 'MLC-EMERGENCY_LIGHTS',
 'MLC-GAME_CONSOLES',
 'MLC-SUPPLEMENTS',
 'MLC-GAME_CONSOLES',
 'MLC-THERMOMETERS',
 'MLC-GAME_CONSOLES',
 'MLC-TOOL_AND_CONSTRUCTION_SUPPLIES',
 'MLC-GAME_CONSOLES',
 'MLC-LAMINATORS',
 'MLC-GAME_CONSOLES',
 'MLC-BODYWEIGHT_SCALES',
 'MLC-GAME_CONSOLES',
 'MLC-CAMPING_AND_FISHING_EQUIPMENT',
 'MLC-GAME_CONSOLES',
 'MLC-HOT_TUBS',
 'MLC-GAME_CONSOLES',
 'MLC-SURGICAL_AND_INDUSTRIAL_MASKS',
 'MLC-GAME_CONSOLES',
 'MLC-CLOTHES_HANGERS',
 'MLC-GAME_CONSOLES',
 'MLC-MATTRESSES',
 'MLC-GAME_CONSOLES',
 'MLC-OFFICE_CHAIRS',
 'MLC-GAME_CONSOLES',
 'MLC-ELECTRIC_CIRCULAR_SAWS',
 'MLC-GAME_CONSOLES',
 'MLC-RESINS',
 'MLC-GAME_CONSOLES',
 'MLC-STATIONARY_BICYCLES',
 'MLC-GAME_CONSOLES',
 'MLC-WELDING_MACHINES',
 'MLC-GAME_CONSOLES',
 'MLC-OFFICE_CHAIRS',
 'MLC-GAME_CONSOLES',
 'MLC-ELECTRIC_BRUSH